**Варіант 1**

**ЗАВДАННЯ 1**

1. Завантажте датасет для рецензій (ml-100k) за допомогою бібліотеки
Surprise.

In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357277 sha256=01a8c009ff381cd34846786f9c2285dee5e3818c7fdffadb78e5d2c5186b2767
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [3]:
from surprise import Dataset, Reader
from surprise import SVD, KNNBasic
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import accuracy
from surprise.model_selection import train_test_split

In [4]:
data = Dataset.load_builtin('ml-100k')

for row in data.raw_ratings[:5]:
    print(row)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k
('196', '242', 3.0, '881250949')
('186', '302', 3.0, '891717742')
('22', '377', 1.0, '878887116')
('244', '51', 2.0, '880606923')
('166', '346', 1.0, '886397596')


2. Виведіть перші 5 рядків завантаженого датасету.

In [5]:
data = Dataset.load_builtin('ml-100k')

print("Перші 5 рядків датасету з поясненням:")
for row in data.raw_ratings[:5]:
    user_id, movie_id, rating, timestamp = row
    print(f"Користувач {user_id} оцінив фільм {movie_id} на {rating} балів (часова позначка: {timestamp})")

Перші 5 рядків датасету з поясненням:
Користувач 196 оцінив фільм 242 на 3.0 балів (часова позначка: 881250949)
Користувач 186 оцінив фільм 302 на 3.0 балів (часова позначка: 891717742)
Користувач 22 оцінив фільм 377 на 1.0 балів (часова позначка: 878887116)
Користувач 244 оцінив фільм 51 на 2.0 балів (часова позначка: 880606923)
Користувач 166 оцінив фільм 346 на 1.0 балів (часова позначка: 886397596)


3. Реалізуйте два алгоритми для рекомендаційної системи на основі цього
датасету. Можна вибрати будь-які алгоритми з бібліотеки Surprise.

In [6]:
trainset, testset = train_test_split(data, test_size=0.2)

param_grid_svd = {
    'n_factors': [50, 100],
    'lr_all': [0.005, 0.010],
    'reg_all': [0.02, 0.05]
}

param_grid_knn = {
    'k': [20, 40],
    'min_k': [1, 5],
    'sim_options': {
        'name': ['cosine', 'pearson'],
        'user_based': [False, True]
    }
}

grid_search_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'], cv=5)
grid_search_svd.fit(data)

grid_search_knn = GridSearchCV(KNNBasic, param_grid_knn, measures=['rmse', 'mae'], cv=5)
grid_search_knn.fit(data)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing

4. Використайте крос-валідацію для підбору оптимальних параметрів для
обох алгоритмів

In [8]:
best_svd = grid_search_svd.best_estimator['mae']
best_knn = grid_search_knn.best_estimator['mae']

print(f"Найкращий MAE для SVD: {grid_search_svd.best_score['mae']}")
print(f"Найкращий MAE для KNNBasic: {grid_search_knn.best_score['mae']}")

Найкращий MAE для SVD: 0.72460120896715
Найкращий MAE для KNNBasic: 0.8029440975080033


5. Оберіть найкращий алгоритм на основі середньої абсолютної помилки
(MAE).

In [9]:
if grid_search_svd.best_score['mae'] < grid_search_knn.best_score['mae']:
    best_model = best_svd
    print("Вибраний алгоритм: SVD")
else:
    best_model = best_knn
    print("Вибраний алгоритм: KNNBasic")

Вибраний алгоритм: SVD


6. Виведи рекомендації (10 фільмів) для конкретного користувача.

In [10]:
best_model.fit(trainset)

user_id = str(153)
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
print(f"Кількість рецензій користувача {user_id}: {len(user_ratings)}")

all_items = set(trainset.all_items())
rated_items = set([item for (item, _) in user_ratings])
unrated_items = all_items - rated_items

predictions = [(item, best_model.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key=lambda x: x[1], reverse=True)

print("Топ-10 фільмів, рекомендованих для користувача:")
for item_id, rating in predictions[:10]:
    print(f"Фільм {trainset.to_raw_iid(item_id)} з прогнозованим рейтингом {rating:.2f}")

Кількість рецензій користувача 153: 20
Топ-10 фільмів, рекомендованих для користувача:
Фільм 1449 з прогнозованим рейтингом 4.30
Фільм 286 з прогнозованим рейтингом 4.02
Фільм 663 з прогнозованим рейтингом 3.98
Фільм 60 з прогнозованим рейтингом 3.98
Фільм 1524 з прогнозованим рейтингом 3.95
Фільм 923 з прогнозованим рейтингом 3.93
Фільм 114 з прогнозованим рейтингом 3.88
Фільм 165 з прогнозованим рейтингом 3.87
Фільм 169 з прогнозованим рейтингом 3.83
Фільм 474 з прогнозованим рейтингом 3.82


**ВИСНОВОК**

Завантажили датасет для рецензій за допомогою бібліотеки Surprise. За допомогою крос-валідації підібрали оптимальні параметри для обох алгоритмів і визначили найкращі на основі середньої абсолютної помилки.

Найкращий MAE для SVD: 0.72460120896715

Найкращий MAE для KNNBasic: 0.8029440975080033

Підсумком вивели 10 рекомендацій фільмів для користувача.

**ЗАВДАННЯ 2**

1. Побудуйте власну рекомендаційну систему (приклад є в документації).
Оцініть її.
2. Отримайте рекомендацію для певного користувача.

In [17]:
model = SVD()
model.fit(trainset)
predictions = model.test(testset)

accuracy.rmse(predictions)

def get_top_n_for_user(predictions, uid, n=15):
    user_ratings = []

    for prediction in predictions:
        if prediction.uid == uid:
            user_ratings.append((prediction.iid, prediction.est))
    user_ratings.sort(key=lambda x: x[1], reverse=True)

    return user_ratings[:n]

specific_user_id = '11'

top_n_recommendations_for_user = get_top_n_for_user(predictions, specific_user_id, n=15)

print(f"Рекомендації для користувача {specific_user_id}:")
for (iid, rating) in top_n_recommendations_for_user:
    print(f"Фільм {iid} з передбаченим рейтингом {rating:.2f}")

RMSE: 0.9280
Рекомендації для користувача 11:
Фільм 654 з передбаченим рейтингом 4.26
Фільм 57 з передбаченим рейтингом 4.10
Фільм 707 з передбаченим рейтингом 4.08
Фільм 659 з передбаченим рейтингом 4.06
Фільм 430 з передбаченим рейтингом 4.01
Фільм 652 з передбаченим рейтингом 4.00
Фільм 435 з передбаченим рейтингом 3.91
Фільм 185 з передбаченим рейтингом 3.90
Фільм 100 з передбаченим рейтингом 3.85
Фільм 750 з передбаченим рейтингом 3.82
Фільм 208 з передбаченим рейтингом 3.80
Фільм 425 з передбаченим рейтингом 3.78
Фільм 746 з передбаченим рейтингом 3.78
Фільм 175 з передбаченим рейтингом 3.77
Фільм 222 з передбаченим рейтингом 3.63


**ВИСНОВОК**

Побудовано власну рекомендаційну систему, оцінено її та виведено рекомендації фільмів для користувача.